In [75]:
import pandas as pd
from matplotlib import pyplot as plt
from io import StringIO
from glob import glob
from os import makedirs

In [30]:
countries = """
AGO
CAF
ZAF
MDG
LVA
VNM
THA
SWE
SDN
PHL
PER
POL
LAO
COL
DEU
VEN
BGD
AUS
NPL
NLD
MYS
MMR
IRN
IND
IDN
GBR
FRA
CAN
""".strip("\n").split("\n")
assert(len(countries) == len(set(countries)))

In [19]:
def try_to_int(i):
    result = i
    try:
        result = int(i)
    except:
        pass
    finally:
        return str(result)

def load_xls(f):
    df = pd.read_excel(f)
    df.columns = [try_to_int(i) for i in df.iloc[2].tolist()]
    df = df.iloc[3:].reset_index(drop=True)
    return df

def load_csv(f):
    df = pd.read_csv(f)
    return df

xls = [load_xls(f) for f in glob("data/*.xls")]
csv = [load_csv(f) for f in glob("data/*.csv")]

df_xls = pd.concat(xls)
df_csv = pd.concat(csv)

In [21]:
df_xls.head(5)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,78.900000,83.780000,88.661227,93.542454,97.170000,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,10.126945,12.237716,14.485849,15.753330,17.310987,20.063024,22.589591,24.988401,27.660654,NaN
2,Afghanistan,AFG,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,5.900000,7.000000,8.260000,11.000000,13.500000,16.800000,17.600000,18.400000,NaN,NaN
3,Africa Western and Central,AFW,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,12.577756,14.923441,18.109485,22.869750,27.081219,32.089337,35.276448,41.749819,46.990497,NaN
4,Angola,AGO,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,13.000000,21.400000,22.000000,23.200000,26.000000,29.000000,32.129392,32.550147,32.602302,NaN


In [22]:
df_csv.head(5)

,Id,IndicatorCode,SpatialDimension,SpatialDimensionValueCode,TimeDimension,TimeDim,DisaggregatingDimension1,DisaggregatingDimension1ValueCode,DisaggregatingDimension2,DisaggregatingDimension2ValueCode,...,FactValueNumeric,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,FactValueTranslationID,FactComments,Language,DateModified
0,32306123.0,WHS3_41,REGION,AFR,YEAR,1980.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32306124.0,WHS3_41,REGION,AMR,YEAR,1980.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32306125.0,WHS3_41,REGION,EMR,YEAR,1980.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,32306126.0,WHS3_41,REGION,EUR,YEAR,1980.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,32306127.0,WHS3_41,REGION,GLOBAL,YEAR,1980.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
data = []

for i, row in df_csv.iterrows():
    indicator = row['IndicatorCode']

    if indicator.startswith("WHS3"):
        country = row['SpatialDimensionValueCode']
        year = int(row['TimeDim'])
        val = row['NumericValue']
    else:
        country = row['SpatialDimValueCode']
        year = int(row['Period'])
        val = row['Value']

    data.append([country, indicator, year, val])

year_cols = [i for i in df_xls.columns if i.isdigit()]
for i, row in df_xls.iterrows():
    country = row['Country Code']
    indicator = row['Indicator Code']
    for year_s in year_cols:
        year = int(year_s)
        val = row[year_s]
        if not pd.isna(val):
            data.append([country, indicator, year, val])

df_d = pd.DataFrame(data, columns=['country', 'indicator', 'year', 'val'])
df_d

,country,indicator,year,val
0,AFR,WHS3_41,1980,8771.000000
1,AMR,WHS3_41,1980,5570.000000
2,EMR,WHS3_41,1980,19970.000000
3,EUR,WHS3_41,1980,271.000000
4,GLOBAL,WHS3_41,1980,97164.000000
...,...,...,...,...
83046,ZWE,SP.POP.GROW,2018,2.020537
83047,ZWE,SP.POP.GROW,2019,1.989253
83048,ZWE,SP.POP.GROW,2020,2.031112
83049,ZWE,SP.POP.GROW,2021,2.045715


In [147]:
df_d['indicator'].unique()

array(['WHS3_41', 'WHS3_46', 'WHS3_49', 'WHS3_62', 'WHS4_100', 'WHS4_544',
       'WHS8_110', 'IT.NET.USER.ZS', 'NY.GDP.PCAP.CD', 'SP.POP.GROW'],
      dtype=object)

In [148]:
df = df_d.sort_values(['year', 'indicator', 'country'])
df

,country,indicator,year,val
45876,BDI,IT.NET.USER.ZS,1960,0.0
46430,BWA,IT.NET.USER.ZS,1960,0.0
48186,GRD,IT.NET.USER.ZS,1960,0.0
50048,MDV,IT.NET.USER.ZS,1960,0.0
52260,SYR,IT.NET.USER.ZS,1960,0.0
...,...,...,...,...
41009,WSM,WHS8_110,2022,82.0
40981,YEM,WHS8_110,2022,73.0
41025,ZAF,WHS8_110,2022,86.0
41048,ZMB,WHS8_110,2022,90.0


In [149]:
# df.to_excel("united_full.xlsx")

In [150]:
for cnt in countries:
    if not len(df[df['country'] == cnt]):
        print(cnt, end=";")

In [151]:
df2 = df[df['country'].isin(countries)]
df2

,country,indicator,year,val
52462,THA,IT.NET.USER.ZS,1960,0.000000
54034,AUS,NY.GDP.PCAP.CD,1960,1810.597443
54445,BGD,NY.GDP.PCAP.CD,1960,84.825337
55152,CAF,NY.GDP.PCAP.CD,1960,66.770095
55215,CAN,NY.GDP.PCAP.CD,1960,2259.250511
...,...,...,...,...
41058,SWE,WHS8_110,2022,92.000000
41095,THA,WHS8_110,2022,96.000000
40969,VEN,WHS8_110,2022,68.000000
41032,VNM,WHS8_110,2022,88.000000


In [212]:
df2.to_excel("united.xlsx")

In [153]:
def draw_plot(df2, country_code, indicator_code, country_name, indicator_name, file_name, size=(8, 4)):
    df3 = df2[(df2['country'] == country_code) & (df2['indicator'] == indicator_code)][['year', 'val']]
    ax = pd.DataFrame(df3['val'].tolist(), index=df3['year'].tolist(), columns=[indicator_name]).plot.bar(title=country_name, figsize=size)
    ax.figure.savefig(file_name)
    plt.close()

In [157]:
len(df2['country'].unique()), len(df2['indicator'].unique())

(28, 10)

In [213]:
def missing(df2):
    df4 = pd.DataFrame(columns=sorted(df2['country'].unique()), index=sorted(df2['indicator'].unique()))
    for cnt in df2['country'].unique():    
        for ind in df2['indicator'].unique():
            yrs = df2[(df2['country'] == cnt) & (df2['indicator'] == ind)]['year']
            ymin, ymax, ycnt = yrs.min(), yrs.max(), yrs.count()
            maxcnt = ymax - ymin + 1
            missed_years = maxcnt - ycnt
            val_miss = f" (-{missed_years})" if missed_years else ""
            val = f"{ymin} - {ymax}{val_miss}"
            df4.loc[ind][cnt] = val
    df4.to_excel("missing_years.xlsx")

# missing(df2)

In [184]:
WHS3_NAMES = {
    'WHS3_55': 'Congenital Rubella Syndrome',
    'WHS3_41': 'Diphtheria',
    'WHS3_42': 'Japanese encephalitis',
    'WHS3_62': 'Measles',
    'WHS3_53': 'Mumps',
    'WHS3_56': 'Neonatal tetanus',
    'WHS3_49': 'Poliomyelitis',
    'WHS3_57': 'Rubella',
    'WHS3_46': 'Total tetanus',
    'WHS3_50': 'Yellow fever'
}

ind_names = {}

for ind in df_d['indicator'].unique():
    if ind.startswith("WHS"):
        val = df_csv[df_csv['IndicatorCode'] == ind]['Indicator'][0]
        if pd.isna(val):
            val = WHS3_NAMES[ind]        
    else:
        val = df_xls[df_xls['Indicator Code'] == ind]['Indicator Name'][0]
    ind_names[ind] = val
ind_names

{'WHS3_41': 'Diphtheria',
 'WHS3_46': 'Total tetanus',
 'WHS3_49': 'Poliomyelitis',
 'WHS3_62': 'Measles',
 'WHS4_100': 'Diphtheria tetanus toxoid and pertussis (DTP3) immunization coverage among 1-year-olds (%)',
 'WHS4_544': 'Polio (Pol3) immunization coverage among 1-year-olds (%)',
 'WHS8_110': 'Measles-containing-vaccine first-dose (MCV1) immunization coverage among 1-year-olds (%)',
 'IT.NET.USER.ZS': 'Individuals using the Internet (% of population)',
 'NY.GDP.PCAP.CD': 'GDP per capita (current US$)',
 'SP.POP.GROW': 'Population growth (annual %)'}

In [192]:
cnt_names_df = pd.read_csv("support/country_names.csv")
cnt_names = {code: country for (code, country) in zip(cnt_names_df['code'], cnt_names_df['country'])}
cnt_names

{'ABW': 'Aruba',
 'AFG': 'Afghanistan',
 'AGO': 'Angola',
 'AIA': 'Anguilla',
 'ALA': 'Åland Islands',
 'ALB': 'Albania',
 'AND': 'Andorra',
 'ARE': 'United Arab Emirates',
 'ARG': 'Argentina',
 'ARM': 'Armenia',
 'ASM': 'American Samoa',
 'ATA': 'Antarctica',
 'ATF': 'French Southern Territories',
 'ATG': 'Antigua and Barbuda',
 'AUS': 'Australia',
 'AUT': 'Austria',
 'AZE': 'Azerbaijan',
 'BDI': 'Burundi',
 'BEL': 'Belgium',
 'BEN': 'Benin',
 'BES': 'Bonaire, Sint Eustatius and Saba',
 'BFA': 'Burkina Faso',
 'BGD': 'Bangladesh',
 'BGR': 'Bulgaria',
 'BHR': 'Bahrain',
 'BHS': 'Bahamas',
 'BIH': 'Bosnia and Herzegovina',
 'BLM': 'Saint Barthélemy',
 'BLR': 'Belarus',
 'BLZ': 'Belize',
 'BMU': 'Bermuda',
 'BOL': 'Bolivia, Plurinational State of',
 'BRA': 'Brazil',
 'BRB': 'Barbados',
 'BRN': 'Brunei Darussalam',
 'BTN': 'Bhutan',
 'BVT': 'Bouvet Island',
 'BWA': 'Botswana',
 'CAF': 'Central African Republic',
 'CAN': 'Canada',
 'CCK': 'Cocos (Keeling) Islands',
 'CHE': 'Switzerland',
 

In [209]:
def full_name(what, where, limit=50, how_far=3):
    fname = f"{what} - {where.get(what, what)}"    
    if len(fname) > (limit + how_far):
        fname = f"{fname[:limit]}..{fname[-how_far:]}"
    return fname

In [211]:
%%time

for cnt in df2['country'].unique():
    path = f"plots/{cnt_names.get(cnt, cnt)}"
    makedirs(path, exist_ok=True)
    for ind in df2['indicator'].unique():
        filename = f"{path}/{cnt}_{ind}.png"
        draw_plot(df2, cnt, ind, full_name(cnt, cnt_names), full_name(ind, ind_names), filename)

CPU times: total: 34 s
Wall time: 48.9 s
